In [2]:
import re
import pandas as pd
import statsmodels.api as sm

In [3]:
metadata = pd.read_csv('meta.csv')
metadata.drop(['price', 'imUrl', 'salesRank', 'related'], axis = 1, inplace = True)

ratings = pd.read_csv('ratings_Musical_Instruments.csv', names = ['reviewerID', 'asin', 'rating', 'unixReviewTime'])
del ratings['unixReviewTime']

df = pd.merge(ratings, metadata, on = 'asin')
df.drop(['reviewerID', 'asin'], axis = 1, inplace = True)

In [4]:
guitarOnly = df[df['title'].str.contains('guitar', na = False)]
guitarOnly = guitarOnly[df['categories'].apply(str).str.contains('Guitars' or 'Stringed')]
guitarOnly['titleDesc'] = guitarOnly['title'] + ' ' + guitarOnly['description']
guitarOnly.drop(['title', 'categories', 'brand', 'description'], axis = 1, inplace = True)
guitarOnly = guitarOnly.dropna().reset_index(drop = True)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [27]:
titleDescFixed = []
regex = re.compile('[^a-zA-Z]')
link = '/Users/renswny/Code/R/MIS584/Final Project/stopwords/english'
with open(link) as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

for i in range(len(guitarOnly)):
    temp = regex.sub(' ', guitarOnly.iloc[i,1])
    temp = temp.lower()
    temp = ' '.join([word for word in temp.split() if word not in stopwords])
    titleDescFixed.append(temp)
    
guitarOnly['titleDescFixed'] = titleDescFixed
del guitarOnly['titleDesc']

In [57]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
data_corpus = list(guitarOnly.iloc[:,1])

X = vectorizer.fit_transform(data_corpus)
bagOfWords = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names())

In [62]:
lm = sm.OLS(guitarOnly['rating'], bagOfWords).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.478
Model:                            OLS   Adj. R-squared:                 -0.156
Method:                 Least Squares   F-statistic:                    0.7532
Date:                Sat, 07 Apr 2018   Prob (F-statistic):              0.714
Time:                        14:15:37   Log-Likelihood:                -36.995
No. Observations:                  32   AIC:                             110.0
Df Residuals:                      14   BIC:                             136.4
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
achieved                  

In [100]:
posResults = pd.DataFrame({'coefficient': lm.params[(lm.pvalues < 0.05) & (lm.params > 0)]})
negResults = pd.DataFrame({'coefficient': lm.params[(lm.pvalues < 0.05) & (lm.params < 0)]})
print(posResults.sort_values(by = ['coefficient'], ascending = False).to_string())
print(negResults.sort_values(by = ['coefficient']).to_string())

             coefficient
guitar          0.576236
ultra           0.499729
acoustic        0.387961
body            0.340722
semi            0.293131
equipped        0.270820
hollow          0.251478
epiphone        0.249865
ultimate        0.249865
technology      0.249865
packed          0.249865
designed        0.247284
black           0.245089
guitars         0.235284
new             0.166790
cone            0.163972
solid           0.156563
mahogany        0.156252
specially       0.144637
demand          0.139346
providing       0.139346
models          0.139346
sweet           0.139346
wound           0.139346
rocker          0.139346
yorktown        0.139346
bv              0.139346
cort            0.139346
alnico          0.139346
left            0.122292
orchestra       0.122292
vaom            0.122292
folding         0.122292
lh              0.122292
handed          0.122292
case            0.120003
style           0.116193
brass           0.109315
bout            0.105122
